# NZ Admin jobs data exploration

## 1. Load data and overview

In [1]:
import pandas as pd
import re

In [2]:
# load file
df = pd.read_excel('NZ_Admin_JOBS.xlsx')

In [3]:
# get info
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   字段1       2708 non-null   object
 1   字段1_link  2708 non-null   object
 2   字段2       2686 non-null   object
 3   字段3       2708 non-null   object
 4   字段4       2708 non-null   object
 5   字段5       2708 non-null   object
dtypes: object(6)
memory usage: 127.1+ KB


#### Total 2,708 observations, 6 columns
#### No column name, all data are object

In [4]:
df.describe()

,字段1,字段1_link,字段2,字段3,字段4,字段5
count,2708,2708,2686,2708,2708,2708
unique,548,2708,475,147,54,93
top,Executive Assistant,https://www.seek.co.nz/job/50582301?type=promo...,Beyond Recruitment - Winner – Best Innovation ...,location: AucklandAucklandarea: Auckland Centr...,"27d ago,at",classification: Administration & Office Suppor...
freq,301,1,285,654,572,715


#### '字段1_link' could be index
#### 22 Missing value in '字段2'
#### Column '字段1' contains position
#### Column '字段1_link'contains link
#### Column '字段2'  contains company name
#### Column '字段3' contains location
#### Column '字段4' shows release time
#### Column '字段5' contains position classification

In [5]:
df.head(5)

,字段1,字段1_link,字段2,字段3,字段4,字段5
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at",classification: Administration & Office Suppor...
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at",classification: Administration & Office Suppor...
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...


### Poor data quality
#### 1. No column name;
#### 2. No index;
#### 3. Null value;
#### 4. Duplicate value;
#### 5. Upper case and lower case mixed;
#### 6. Abbreviation and full name mixed;
#### 7. Time data is not clear
#### 8. Missing data
#### 9. Data mixed in one column
#### 10. Salary format is different;
#### 11. Classification is too long. Parent classification are the same.

## 2. Data preprocess

#### 2.1 Add column name

In [6]:
df_copy = df.copy()

In [7]:
# add column name first
column_name = ['Position','Link','Company','Location','Release_time','Classification']
df_copy.columns = column_name
df_copy.head(1)

,Position,Link,Company,Location,Release_time,Classification
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...


#### 2.2 Clean column 'Position'

#### 'office administrator' and 'Office Administrator' should be the same position

#### 2.2.1 Capitalize each word in Column 'Position'

In [8]:
# Capitalize the value
df_copy['Position'] = df_copy['Position'].str.title()
df_copy.head(5)

,Position,Link,Company,Location,Release_time,Classification
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at",classification: Administration & Office Suppor...
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at",classification: Administration & Office Suppor...
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...


#### Column 'Position' inlcude work type description, like 'full time','part time','10hours per week','temp'...
#### 2.2.2 Create and fill in Column 'Work type'

In [9]:
# select work type
print(df_copy['Position'].str.contains("[Ff]ull").sum()) #5
print(df_copy['Position'].str.contains("[Pp]art").sum()) #41
print(df_copy['Position'].str.contains("[Ff]ix").sum()) #20
print(df_copy['Position'].str.contains("[Pp]er").sum()) #308
print(df_copy['Position'].str.contains("[Cc]ontract").sum()) #96
df_copy['Worktype'] = df_copy.loc[df_copy['Position'].str.contains("[Ff]ull|[Pp]art|[Ff]ix|[Pp]er|[Cc]ontract"),'Position']


5
41
20
308
96


#### 2.2.3 Clean column 'Link'
#### job/50582301? number between'/' and '?' seems company ID

In [10]:
# filter companyID from Link

pattern = r"\d{8}"
df_copy['CompanyID'] = df_copy['Link'].apply(lambda x: re.findall(pattern,x)[0])
df_copy.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50582301


#### 2.2.4 Clean column 'Company' 

In [11]:
# To find 22 missing value records in Column 'Company'
df_copy[df_copy['Company'].isnull()].head(5)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50582301
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50622432
19,Administrator,https://www.seek.co.nz/job/50604829?type=stand...,NaN,location: ManawatuManawatuarea: Rest of Manawa...,"7d ago,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50604829
60,Administrator,https://www.seek.co.nz/job/50582301?type=stand...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"11d ago,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50582301
64,Administration Specialist - Construction,https://www.seek.co.nz/job/50638765?type=stand...,NaN,location: AucklandAucklandarea: Rodney & North...,"5h ago,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50638765


#### I found that 22 null in company because they are private advertisers
#### Next, I will fill "Private Advertiser" in them

In [12]:
# fill null value by 'Private Advertiser'
df_copy['Company'] = df_copy['Company'].fillna(value = 'Private Advertiser')
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Position        2708 non-null   object
 1   Link            2708 non-null   object
 2   Company         2708 non-null   object
 3   Location        2708 non-null   object
 4   Release_time    2708 non-null   object
 5   Classification  2708 non-null   object
 6   Worktype        454 non-null    object
 7   CompanyID       2708 non-null   object
dtypes: object(8)
memory usage: 169.4+ KB


#### 2.2.5 Deal with column 'Location'
#### inlcude wage data, extra word like"location:"

In [13]:
# check number of wage data inlcuded in Column'Location'
print(df_copy['Location'].str.contains(',').sum()) # 603

603


#### 603 Location data include wage

In [14]:
# delete wage data in Column 'Locaiton'
df_copy['Location'] = df_copy['Location'].apply(lambda x: x.split(',')[0] if "," in x else x)

In [15]:
print(df_copy['Location'].str.contains(',').sum()) 

0


In [16]:
# create a column include salary
df_copy['Salary'] = df_copy['Location'].apply(lambda x: x.split(',')[1] if "," in x else "")

In [17]:
 # remove extra word "location:"
df_copy['Location'] = df_copy['Location'].apply(lambda x: x[9:])

In [18]:
df_copy['Area'] = df_copy['Location'].apply(lambda x: x.split(':')[0])
df_copy.head(5)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50582301,,Bay of PlentyBay of Plentyarea
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at",classification: Administration & Office Suppor...,NaN,50620889,,Bay of PlentyBay of Plentyarea
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...,NaN,50622169,,AucklandAuckland
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea: InvercargillInvercargill,"1h ago,at",classification: Administration & Office Suppor...,NaN,50639620,,SouthlandSouthlandarea
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Private Advertiser,CanterburyCanterburyarea: ChristchurchChristc...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50622432,,CanterburyCanterburyarea


In [ ]:
# split and drop repeat word
def drop_repeat(s):
    for x in range(1, len(s)):
        suffix = s[-x:]
        if s.startswith(suffix):
            return (s[:-x], suffix)

In [19]:
df_copy.head(5)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50582301,,Bay of PlentyBay of Plentyarea
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at",classification: Administration & Office Suppor...,NaN,50620889,,Bay of PlentyBay of Plentyarea
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...,NaN,50622169,,AucklandAuckland
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea: InvercargillInvercargill,"1h ago,at",classification: Administration & Office Suppor...,NaN,50639620,,SouthlandSouthlandarea
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Private Advertiser,CanterburyCanterburyarea: ChristchurchChristc...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...,NaN,50622432,,CanterburyCanterburyarea


#### 2.2.6 Deal with column 'Release_time'
#### non useful word like 'at', 'Private Advertiser' need be deleted

In [20]:
df_copy['Release_time'] = df_copy['Release_time'].apply(lambda x: x.split(',')[0])


In [21]:
df_copy.head(5)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,Featured,classification: Administration & Office Suppor...,NaN,50582301,,Bay of PlentyBay of Plentyarea
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,Featured,classification: Administration & Office Suppor...,NaN,50620889,,Bay of PlentyBay of Plentyarea
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,4d ago,classification: Administration & Office Suppor...,NaN,50622169,,AucklandAuckland
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea: InvercargillInvercargill,1h ago,classification: Administration & Office Suppor...,NaN,50639620,,SouthlandSouthlandarea
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Private Advertiser,CanterburyCanterburyarea: ChristchurchChristc...,4d ago,classification: Administration & Office Suppor...,NaN,50622432,,CanterburyCanterburyarea


#### 2.2.7 Deal with Column Classification

In [22]:
print(df_copy['Classification'].str.contains("\d+").sum()) # 96 rows were filled by salary


96


In [23]:
df_copy['Classification'] = df_copy['Classification'].apply(lambda x: x.split(':')[1] if ":" in x else "")

In [24]:
df_copy.head(5)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,Featured,Administration & Office SupportAdministration...,NaN,50582301,,Bay of PlentyBay of Plentyarea
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,Featured,Administration & Office SupportAdministration...,NaN,50620889,,Bay of PlentyBay of Plentyarea
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,4d ago,Administration & Office SupportAdministration...,NaN,50622169,,AucklandAuckland
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea: InvercargillInvercargill,1h ago,Administration & Office SupportAdministration...,NaN,50639620,,SouthlandSouthlandarea
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Private Advertiser,CanterburyCanterburyarea: ChristchurchChristc...,4d ago,Administration & Office SupportAdministration...,NaN,50622432,,CanterburyCanterburyarea


In [25]:
df_copy.to_excel("NZ_Admin_JOBS_1.xlsx")